# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

`Суть проекта` - Даны результаты проб нефти в трёх регионах, в каждом из регионов - 10 тысяч месторождений. Нужно проанализировать возможную прибыль и риски.

`Конечная цель проекта` - Выбрать наилучший регион для разработки скважин

Начнем с импортирования нужные библиотек и открытии данных и изучение их

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
display(geo_data_0.head())
geo_data_0.info()
geo_data_0.describe()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [4]:
display(geo_data_1.head())
geo_data_1.info()
geo_data_1.describe()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [5]:
display(geo_data_2.head())
geo_data_2.info()
geo_data_2.describe()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


По всем 3-ем датасетам пропусков нет и типы данных менять не нужно.

## Обучение и проверка модели

Начнем с первого региона и разобьем данные на обучающая и валидационную выборку:

In [6]:
features_valid, features_test, target_valid, target_test = train_test_split(geo_data_0[['f0', 'f1', 'f2']],geo_data_0['product'], test_size=0.25, random_state=12345)
print(features_valid.shape, features_test.shape, target_valid.shape, target_test.shape)

(75000, 3) (25000, 3) (75000,) (25000,)


Всё отлично разделилось. 

Далее обучим модель и сделаем предсказания на валидационной выборке

In [7]:
model = LinearRegression()
model.fit(features_valid, target_valid)
predicted = model.predict(features_test)

Далее сохраним предсказания и правильные ответы на валидационной выборке.

In [8]:
region1 = pd.DataFrame(target_test)
region1['predicted'] = predicted
region1['region'] = 1
display(region1)

,product,predicted,region
71751,10.038645,95.894952,1
80493,114.551489,77.572583,1
2655,132.603635,77.892640,1
53233,169.072125,90.175134,1
91141,122.325180,70.510088,1
...,...,...,...
12581,170.116726,103.037104,1
18456,93.632175,85.403255,1
73035,127.352259,61.509833,1
63834,99.782700,118.180397,1


Таблица создана, теперь надо напечать на экран средний запас предсказанного сырья и RMSE модели:

In [9]:
print(region1[['product', 'predicted']].mean())
mse = mean_squared_error(target_test, predicted)
rmse = np.sqrt(mse)
print('RMSE = ', rmse)

product      92.078597
predicted    92.592568
dtype: float64
RMSE =  37.5794217150813


По итогу в первом регионе предсказано 92.59 сырья, при этом на деле 92.07, а RMSE модели равно 37.57

Далее посмотрим второй регион:

In [10]:
features_valid, features_test, target_valid, target_test = train_test_split(geo_data_1[['f0', 'f1', 'f2']],geo_data_1['product'], test_size=0.25, random_state=12345)
print(features_valid.shape, features_test.shape, target_valid.shape, target_test.shape)

(75000, 3) (25000, 3) (75000,) (25000,)


In [11]:
model.fit(features_valid, target_valid)
predicted = model.predict(features_test)
region2 = pd.DataFrame(target_test)
region2['predicted'] = predicted
region2['region'] = 2
display(region2)

,product,predicted,region
71751,80.859783,82.663314,2
80493,53.906522,54.431786,2
2655,30.132364,29.748760,2
53233,53.906522,53.552133,2
91141,0.000000,1.243856,2
...,...,...,...
12581,137.945408,136.869211,2
18456,110.992147,110.693465,2
73035,137.945408,137.879341,2
63834,84.038886,83.761966,2


In [12]:
print(region2[['product', 'predicted']].mean())
mse = mean_squared_error(target_test, predicted)
rmse = np.sqrt(mse)
print('RMSE = ', rmse)

product      68.723136
predicted    68.728547
dtype: float64
RMSE =  0.893099286775617


По второму региону ситуация следующая:
как предсказано так и самого сырья на деле практически одинаково и RMSE очень низкий.

Далее рассмотрим 3 регион:

In [13]:
features_valid, features_test, target_valid, target_test = train_test_split(geo_data_2[['f0', 'f1', 'f2']], geo_data_2['product'], test_size=0.25, random_state=12345)
features_valid.shape, features_test.shape, target_valid.shape, target_test.shape

((75000, 3), (25000, 3), (75000,), (25000,))

In [14]:
model.fit(features_valid, target_valid)
predicted = model.predict(features_test)
region3 = pd.DataFrame(target_test)
region3['predicted'] = predicted
region3['region'] = 3
display(region3)

,product,predicted,region
71751,61.212375,93.599633,3
80493,41.850118,75.105159,3
2655,57.776581,90.066809,3
53233,100.053761,105.162375,3
91141,109.897122,115.303310,3
...,...,...,...
12581,28.492402,78.765887,3
18456,21.431303,95.603394,3
73035,125.487229,99.407281,3
63834,99.422903,77.779912,3


In [15]:
print(region3[['product', 'predicted']].mean())
mse = mean_squared_error(target_test, predicted)
rmse = np.sqrt(mse)
print('RMSE = ', rmse)

product      94.884233
predicted    94.965046
dtype: float64
RMSE =  40.02970873393434


С третьим регионом ситцация следующая : Запасов сырья немного меньше чем предсказано, однако RMSE Высокий.

#### Вывод

После проведенного исследования, можно прийти к выводу, что больше всего запасов сырья в третьем регионе, но у него же и самый высокий RMSE, при этом меньше всего запасов сырья у второго региона, зато у него же и самый низкий RMSE,

## Подготовка к расчёту прибыли

Укажем бюджет региона равным 10 млрд. рублей

In [16]:
budget = 10000000000

Из 500 точек выберут 200 самых лучших на которые и уйдет весь бюджет по-ровну:

In [17]:
best_budget = budget / 200
print(best_budget)

50000000.0


Объем указан в тысячах баррелях, поэтому доход с 1 единицы баррели будет равен:

In [18]:
barrel_income = 450*1000

Далее надо рассчитать достаточный объем сырья для безубыточной разработки новой скважины и затем сравнить полученный объем сырья со средним запасом в каждом регионе:

In [19]:
break_even_volume = best_budget / barrel_income
if break_even_volume > region1['predicted'].mean():
    print('1-ый регион не имеет достаточного кол-ва сырья')
else:
    print('1-ый регион имеет достаточное кол-во сырья')
if break_even_volume > region2['predicted'].mean():
    print('2-ый регион не имеет достаточного кол-ва сырья')
else:
    print('2-ый регион имеет достаточное кол-во сырья')
if break_even_volume > region3['predicted'].mean():
    print('3-ый регион не имеет достаточного кол-ва сырья')
else:
    print('3-ый регион имеет достаточное кол-во сырья')
print(break_even_volume, '- сколько надо\n')
print('Сколько имеется:', region1['predicted'].mean(), region2['predicted'].mean(), region3['predicted'].mean())

1-ый регион не имеет достаточного кол-ва сырья
2-ый регион не имеет достаточного кол-ва сырья
3-ый регион не имеет достаточного кол-ва сырья
111.11111111111111 - сколько надо

Сколько имеется: 92.59256778438035 68.728546895446 94.96504596800489


##### Вывод
Ни один из регионов не имеет достаточного объема сырья для безубыточной разработки новых скважин.

Далее надо написать функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели, при этом скважины должны быть с максимальным значением предсказаний, затем надо просуммировать целевое значение объема сырья, соответствующее этим предсказаниям и потом рассчитать прибыль для полученного объема сырья.

In [20]:
def profit(region):
    return sum(region.sort_values(by='predicted', ascending=False)['product'].head(200)) * barrel_income

## Расчёт прибыли и рисков 

Теперь воспользуемся техникой bootstrap с 1000 выборок чтобы найти распределение прибыли а затем найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков:

In [21]:
data_region = {}
for region in [region1, region2, region3]:
    values = []
    for i in range(1000):
        region_subsample = region.sample(n=500, replace=False) 
        values.append(profit(region_subsample))
    values = pd.Series(values)
    data_region['region'+str(region['region'][1])] = [sum(values) / 1000 - budget, values.quantile(0.025) - budget, values.quantile(0.975) - budget, (values - budget < 0).mean()]

In [22]:
pd.DataFrame.from_dict(data_region).T.rename(columns={0:'mean_profit', 1:'2.5%_quantile', 2:'97.5%_quantile', 3:'risk'})

,mean_profit,2.5%_quantile,97.5%_quantile,risk
region1,3.801007e+08,-1.571424e+08,8.968339e+08,0.082
region2,4.568301e+08,8.453048e+07,8.430030e+08,0.014
region3,3.907179e+08,-1.613980e+08,8.766983e+08,0.075


##### Вывод

С учетом рисков лучше всего выбрать второй регион т.к у него и самый низкий уровень риска и при этом даже в случае риска будет прибыль из-за 2.5%-го квантиля.

# Конечный вывод:

Проведя исследование на одной моделе и пришел в итоге к тому, что:
- Регион с самым высоким запасом сырья а так же показателем RMSE - `третий регион`, с самыми низкими - `Второй регион`
- При всём при этом ни один регион не имеет достаточного объёма сырья для безубыточной разработки новых скважин.
- Однако во всём остальном с учетом рисков рекомендуется выбрать `2 регион` поскольку у него самый низкий риск и в случае провала прибыль всё равно будет

##### Ответ для компании:
Рекомендуется 2-ой регион поскольку риск у него низкий и прибыль в случае неудачи так же будет.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован